In [5]:
import numpy as np
import pandas as pd

data = pd.read_csv('/readmission_ISP/rawdata.csv')  #might need to change this here
data.head()

,prvdr_num,FFY2012_DSH,FFY2015_DSH,hospital_name,city,state,zip_code,3YR_2008_2011_AMI_Readmit_Rate,3YR_2012_2015_AMI_Readmit_Rate,AMI_Change,3YR_2008_2011_HF_Readmit_Rate,3YR_2012_2015_HF_Readmit_Rate,HF_Change,3YR_2008_2011_PN_Readmit_Rate,3YR_2012_2015_PN_Readmit_Rate,PN_Change
0,140286,0.0251,0.0401,KISHWAUKEE COMMUNITY HOSPITAL,DEKALB,IL,60115,0.132353,0.139241,0.006888,0.247863,0.229236,-0.018627,0.148649,0.160612,0.011963
1,230269,0.0256,0.0448,WILLIAM BEAUMONT HOSPITAL-TROY,TROY,MI,48085,0.224490,0.215630,-0.008860,0.300842,0.233355,-0.067487,0.247297,0.197456,-0.049841
2,100224,0.0262,0.0316,UNIVERSITY HOSPITAL AND MEDICAL CENTER,TAMARAC,FL,33321,0.279070,0.188406,-0.090664,0.221910,0.271777,0.049867,0.173913,0.132404,-0.041509
3,390026,0.0264,0.0578,CHESTNUT HILL HOSPITAL,PHILADELPHIA,PA,19118,0.283582,0.166667,-0.116915,0.307278,0.250000,-0.057278,0.208333,0.196217,-0.012116
4,390036,0.0274,0.0458,HERITAGE VALLEY BEAVER,BEAVER,PA,15009,0.228723,0.174157,-0.054566,0.252912,0.213508,-0.039404,0.169312,0.156250,-0.013062


In [7]:
data.FFY2015_DSH.quantile([0.25, 0.5, 0.75, 1])  #define quantiles

0.25    0.09315
0.50    0.13910
0.75    0.21145
1.00    0.77740
Name: FFY2015_DSH, dtype: float64

In [8]:
#group rows according to quantile

data.loc[data.FFY2015_DSH <= data.FFY2015_DSH.quantile(0.25), 'quantile'] = '1'

data.loc[(data.FFY2015_DSH > data.FFY2015_DSH.quantile(0.25)) & 
         (data.FFY2015_DSH <= data.FFY2015_DSH.quantile(0.5)), 'quantile'] = '2'

data.loc[(data.FFY2015_DSH > data.FFY2015_DSH.quantile(0.5)) & 
         (data.FFY2015_DSH <= data.FFY2015_DSH.quantile(0.75)), 'quantile'] = '3'

data.loc[(data.FFY2015_DSH > data.FFY2015_DSH.quantile(0.75)) & 
         (data.FFY2015_DSH <= data.FFY2015_DSH.quantile(1)), 'quantile'] = '4'


In [9]:
#display summary statistics
data.groupby('quantile').agg({'FFY2015_DSH':['mean','std'],'AMI_Change': ['mean','std'], 'HF_Change': ['mean','std'], 'PN_Change': ['mean','std']})

FFY2015_DSH           AMI_Change           HF_Change            \
                mean       std       mean       std      mean       std   
quantile                                                                  
1           0.065007  0.018345  -0.031665  0.052974 -0.030251  0.034755   
2           0.117731  0.012121  -0.022615  0.057831 -0.023568  0.036178   
3           0.170089  0.020352  -0.029812  0.050130 -0.029523  0.035599   
4           0.327880  0.101642  -0.028014  0.059090 -0.031032  0.041856   

         PN_Change            
              mean       std  
quantile                      
1        -0.016696  0.029464  
2        -0.017734  0.031720  
3        -0.019987  0.030516  
4        -0.013031  0.039056

In [11]:
#perform one-way anova to see if differences by 2015 DSH ratio quantile are signficant

import scipy.stats as stats

q1 = data.loc[data['quantile']=='1','FFY2015_DSH']
q2 = data.loc[data['quantile']=='2','FFY2015_DSH']
q3 = data.loc[data['quantile']=='3','FFY2015_DSH']
q4 = data.loc[data['quantile']=='4','FFY2015_DSH']


f, p = stats.f_oneway(q1,q2,q3,q4)
print ('One-way ANOVA')
print ('=============')
 
print ('F value:', f)
print ('P value:', p, '\n')

One-way ANOVA
F value: 812.009083809
P value: 7.11747270277e-228 



In [12]:
#perform tukey's test to see pairwise group differences

from statsmodels.stats.multicomp import pairwise_tukeyhsd
print(pairwise_tukeyhsd(data['FFY2015_DSH'], data['quantile']))


Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff lower  upper  reject
-------------------------------------------
  1      2     0.0527  0.0382 0.0672  True 
  1      3     0.1051  0.0905 0.1196  True 
  1      4     0.2629  0.2484 0.2774  True 
  2      3     0.0524  0.0378 0.0669  True 
  2      4     0.2101  0.1956 0.2247  True 
  3      4     0.1578  0.1433 0.1723  True 
-------------------------------------------


Tukey's test shows no differences for all 3 types of readmission rates.

Could this mean patient socioeconomic status is not a significant factor in readmission rates?

In [13]:
print(pairwise_tukeyhsd(data['AMI_Change'], data['quantile']))

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  1      2     0.009    -0.006 0.0241 False 
  1      3     0.0019  -0.0133 0.017  False 
  1      4     0.0037  -0.0114 0.0187 False 
  2      3    -0.0072  -0.0223 0.0079 False 
  2      4    -0.0054  -0.0205 0.0097 False 
  3      4     0.0018  -0.0133 0.0169 False 
--------------------------------------------


In [14]:
print(pairwise_tukeyhsd(data['HF_Change'], data['quantile']))

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  1      2     0.0067  -0.0035 0.0169 False 
  1      3     0.0007  -0.0095 0.0109 False 
  1      4    -0.0008   -0.011 0.0094 False 
  2      3     -0.006  -0.0162 0.0042 False 
  2      4    -0.0075  -0.0176 0.0027 False 
  3      4    -0.0015  -0.0117 0.0087 False 
--------------------------------------------


In [15]:
print(pairwise_tukeyhsd(data['PN_Change'], data['quantile']))

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  1      2     -0.001   -0.01  0.008  False 
  1      3    -0.0033  -0.0123 0.0057 False 
  1      4     0.0037  -0.0053 0.0127 False 
  2      3    -0.0023  -0.0113 0.0068 False 
  2      4     0.0047  -0.0043 0.0137 False 
  3      4     0.007   -0.0021 0.016  False 
--------------------------------------------
